In [3]:
import concurrent.futures
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
import yfinance as yf
from pandas.tseries.offsets import BusinessDay
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import os

env_path = '../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

In [4]:
project = hopsworks.login(project='AMomozZz', api_key_value=HOPSWORKS_API_KEY)

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

today = today.isoformat()
yesterday = yesterday.isoformat()

today, yesterday

2024-12-29 14:35:09,675 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-29 14:35:09,678 INFO: Initializing external client
2024-12-29 14:35:09,679 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-29 14:35:10,808 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163416


('2024-12-29', '2024-12-28')

In [66]:
# Retrieve feature groups
fs = project.get_feature_store()
mr = project.get_model_registry()

gold_fg = fs.get_feature_group(
    name='gold_feature_group',
    version=1,
)
exchange_fg = fs.get_feature_group(
    name='exchange_feature_group',
    version=1,
)
only_exchange_fg = fs.get_feature_group(
    name='only_exchange_feature_group',
    version=1,
)
retrieved_model = mr.get_model(
    name="exchange_rate_xgboost_model",
    version=1,
)

saved_model_dir = retrieved_model.download()

In [67]:
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='8.869148E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [166]:
import requests
import pandas as pd

gold_ticker = 'GC=F'
gold_today_df = yf.download(gold_ticker, today)
gold_today_df = gold_today_df.reset_index()
gold_today_df.columns = gold_today_df.columns.get_level_values(0)

new_columns = ["Date"] + [col + " Price" for col in gold_today_df.columns[1:]]
gold_today_df.columns = new_columns

gold_today_df = gold_today_df[["Date", "Close Price"]]
gold_today_df.columns = gold_today_df.columns.str.lower().str.replace(' ', '_')

display(gold_today_df)
gold_today_df.info()

[*********************100%***********************]  1 of 1 completed


,date,close_price
0,2024-12-27,2636.5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1 non-null      datetime64[ns]
 1   close_price  1 non-null      float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 148.0 bytes


In [47]:
startDate="2000-08-30"
endDate = today
base = "USD"

# Fetch data
url = f"https://api.frankfurter.dev/v1/{startDate}..{endDate}?base={base}"

response = requests.get(url)
data = response.json()

# Convert JSON to DataFrame
today_exchange_df = pd.DataFrame.from_dict(data["rates"], orient="index")

# Add a 'Date' column as the first column
today_exchange_df.index.name = "Date"
today_exchange_df.reset_index(inplace=True)
today_exchange_df.columns = today_exchange_df.columns.str.lower()

today_exchange_df = today_exchange_df[["date","aud","bgn","cad","chf","czk","dkk","eur","gbp","hkd","huf","jpy","krw","nok","nzd","pln","sek","sgd","zar"]].tail(11)

today_exchange_df['date'] = pd.to_datetime(today_exchange_df['date'])

today_exchange_df

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,huf,jpy,krw,nok,nzd,pln,sek,sgd,zar
6217,2024-12-11,1.5728,1.8614,1.4186,0.88322,23.905,7.0976,0.95175,0.78479,7.7736,389.72,152.61,1430.36,11.1469,1.7303,4.0585,10.9608,1.3433,17.7781
6218,2024-12-12,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,389.33,152.13,1430.72,11.1086,1.7266,4.0757,10.9618,1.3448,17.6939
6219,2024-12-13,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866
6220,2024-12-16,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,389.63,154.06,1437.56,11.1602,1.7326,4.0617,10.9107,1.3501,17.8788
6221,2024-12-17,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,390.06,153.98,1436.72,11.2065,1.7366,4.0616,10.9288,1.3510,18.1286
6222,2024-12-18,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,391.46,153.89,1438.15,11.2062,1.7438,4.0566,10.9466,1.3507,18.0642
6223,2024-12-19,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,400.24,156.87,1448.10,11.3654,1.7688,4.0885,11.0269,1.3594,18.3117
6224,2024-12-20,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,398.63,156.78,1446.29,11.4230,1.7753,4.1023,11.0452,1.3571,18.3931
6225,2024-12-23,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,399.02,157.18,1453.44,11.3605,1.7732,4.1136,11.0497,1.3570,18.4780
6226,2024-12-24,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,396.08,157.05,1458.37,11.3987,1.7729,4.1092,11.0952,1.3592,18.5948


In [31]:
only_exchange_tail9 = only_exchange_fg.read().sort_values(by=['date'], ascending=True).tail(9)
only_exchange_tail9['date'] = pd.to_datetime(only_exchange_tail9['date']).dt.tz_localize(None)
only_exchange_tail9

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.58s) 


,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,hkd_10,huf_10,jpy_10,krw_10,nok_10,nzd_10,pln_10,sek_10,sgd_10,zar_10
1783,2024-12-13,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,...,7.7837,389.84,150.20,1397.57,11.0590,1.6922,4.0674,10.9051,1.3406,18.0626
5469,2024-12-16,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,...,7.7808,394.93,150.13,1404.33,11.0964,1.6981,4.0835,10.9708,1.3456,18.1820
2150,2024-12-17,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,...,7.7833,394.45,149.64,1403.06,11.0626,1.7002,4.0880,11.0117,1.3451,18.1035
1452,2024-12-18,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,...,7.7840,394.96,151.15,1417.54,11.0756,1.7144,4.0858,11.0065,1.3472,18.1738
5183,2024-12-19,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,...,7.7824,390.70,150.40,1418.04,11.0626,1.7064,4.0555,10.9198,1.3409,18.0671
5730,2024-12-20,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,...,7.7794,391.60,150.51,1419.74,11.0996,1.7118,4.0327,10.8903,1.3396,18.0584
754,2024-12-23,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,...,7.7748,389.15,150.83,1432.14,11.1123,1.7061,4.0304,10.9160,1.3396,17.8194
641,2024-12-24,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,...,7.7760,390.57,151.74,1431.85,11.1442,1.7178,4.0541,10.9561,1.3411,17.8782
1202,2024-12-27,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,...,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866


In [17]:
df_tail10 = exchange_fg.read().sort_values(by=['date'], ascending=True).tail(10)
df_tail10

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.52s) 


,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,close_price_1,close_price_2,close_price_3,close_price_4,close_price_5,close_price_6,close_price_7,close_price_8,close_price_9,close_price_10
2997,2024-12-12 00:00:00+00:00,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,...,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902,2657.000000,0.000000
3004,2024-12-13 00:00:00+00:00,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,...,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902,2657.000000
5469,2024-12-16 00:00:00+00:00,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,...,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902
3371,2024-12-17 00:00:00+00:00,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,...,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951
2673,2024-12-18 00:00:00+00:00,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,...,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049
5183,2024-12-19 00:00:00+00:00,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,...,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098
5730,2024-12-20 00:00:00+00:00,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,...,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098
1975,2024-12-23 00:00:00+00:00,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,...,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902
1862,2024-12-24 00:00:00+00:00,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,...,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098
2423,2024-12-27 00:00:00+00:00,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,...,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000


In [68]:
today_exchange_df['date'].values[10] not in only_exchange_tail9['date'].values

False

In [61]:
result = pd.DataFrame()

if today_exchange_df['date'].values[10] not in only_exchange_tail9['date'].values:
    df_only_exchange_today = today_exchange_df.copy()
    result = today_exchange_df.copy()
    for i in range(1, 11):
        df_temp = df_only_exchange_today.copy()
        df_temp['date'] = pd.to_datetime(df_temp['date']) + BusinessDay(i)
        df_temp = df_temp.rename(columns=lambda x: f"{x.lower()}_{i}" if x != "date" else x)
        result = pd.merge(result, df_temp, how='left', on="date")

    result.fillna(0, inplace=True)
    result = result.tail(1)

result

""


In [64]:
if not result.empty:
    only_exchange_fg.insert(result, write_options={"wait_for_job": True})

In [ ]:
updated_only_exchange_df_tail10 = only_exchange_fg.read().sort_values(by=['date'], ascending=True).tail(10)
updated_only_exchange_df_tail10['date'] = pd.to_datetime(updated_only_exchange_df_tail10['date']).dt.tz_localize(None)
updated_only_exchange_df_tail10

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.80s) 


,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,hkd_10,huf_10,jpy_10,krw_10,nok_10,nzd_10,pln_10,sek_10,sgd_10,zar_10
1776,2024-12-12,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,...,7.7828,391.61,151.67,1396.59,11.0643,1.6991,4.0884,10.9443,1.3437,18.1522
1783,2024-12-13,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,...,7.7837,389.84,150.20,1397.57,11.0590,1.6922,4.0674,10.9051,1.3406,18.0626
5469,2024-12-16,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,...,7.7808,394.93,150.13,1404.33,11.0964,1.6981,4.0835,10.9708,1.3456,18.1820
2150,2024-12-17,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,...,7.7833,394.45,149.64,1403.06,11.0626,1.7002,4.0880,11.0117,1.3451,18.1035
1452,2024-12-18,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,...,7.7840,394.96,151.15,1417.54,11.0756,1.7144,4.0858,11.0065,1.3472,18.1738
5183,2024-12-19,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,...,7.7824,390.70,150.40,1418.04,11.0626,1.7064,4.0555,10.9198,1.3409,18.0671
5730,2024-12-20,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,...,7.7794,391.60,150.51,1419.74,11.0996,1.7118,4.0327,10.8903,1.3396,18.0584
754,2024-12-23,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,...,7.7748,389.15,150.83,1432.14,11.1123,1.7061,4.0304,10.9160,1.3396,17.8194
641,2024-12-24,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,...,7.7760,390.57,151.74,1431.85,11.1442,1.7178,4.0541,10.9561,1.3411,17.8782
1202,2024-12-27,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,...,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866


In [97]:
updated_only_exchange_df_tail10

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,hkd_10,huf_10,jpy_10,krw_10,nok_10,nzd_10,pln_10,sek_10,sgd_10,zar_10
1776,2024-12-12,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,...,7.7828,391.61,151.67,1396.59,11.0643,1.6991,4.0884,10.9443,1.3437,18.1522
1783,2024-12-13,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,...,7.7837,389.84,150.20,1397.57,11.0590,1.6922,4.0674,10.9051,1.3406,18.0626
5469,2024-12-16,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,...,7.7808,394.93,150.13,1404.33,11.0964,1.6981,4.0835,10.9708,1.3456,18.1820
2150,2024-12-17,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,...,7.7833,394.45,149.64,1403.06,11.0626,1.7002,4.0880,11.0117,1.3451,18.1035
1452,2024-12-18,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,...,7.7840,394.96,151.15,1417.54,11.0756,1.7144,4.0858,11.0065,1.3472,18.1738
5183,2024-12-19,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,...,7.7824,390.70,150.40,1418.04,11.0626,1.7064,4.0555,10.9198,1.3409,18.0671
5730,2024-12-20,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,...,7.7794,391.60,150.51,1419.74,11.0996,1.7118,4.0327,10.8903,1.3396,18.0584
754,2024-12-23,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,...,7.7748,389.15,150.83,1432.14,11.1123,1.7061,4.0304,10.9160,1.3396,17.8194
641,2024-12-24,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,...,7.7760,390.57,151.74,1431.85,11.1442,1.7178,4.0541,10.9561,1.3411,17.8782
1202,2024-12-27,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,...,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866


In [146]:
last_date = updated_only_exchange_df_tail10["date"].values[len(updated_only_exchange_df_tail10) - 1]
filtered_columns = [col for col in updated_only_exchange_df_tail10 if "_" not in col and col != "date"]
intermediate_result = updated_only_exchange_df_tail10.copy()

for i in range(1, 6):
    now_date = last_date + BusinessDay(i)
    pred_featrue = intermediate_result.drop(["date"] + filtered_columns, axis=1).tail(1)
    y_pred = retrieved_xgboost_model.predict(pred_featrue)

    y_pred_df = pd.DataFrame(y_pred, columns=filtered_columns)
    y_pred_df["date"] = now_date

    intermediate_result = pd.concat([intermediate_result, y_pred_df], ignore_index=True)
    intermediate_result
    for i in range(1, 11):
        df_temp = intermediate_result.copy()
        df_temp['date'] = pd.to_datetime(df_temp['date']) + BusinessDay(i)
        df_temp = df_temp.loc[df_temp['date'] == now_date, ["date"] + filtered_columns]
        if df_temp.empty:
            new_row = {col: 0 for col in df_temp.columns if col != 'date'}
            new_row['date'] = now_date
            df_temp = pd.concat([df_temp, pd.DataFrame([new_row])], ignore_index=True)
        df_temp = df_temp.rename(columns=lambda x: f"{x.lower()}_{i}" if x != "date" else x)
        row_index = intermediate_result.index[intermediate_result['date'] == now_date]
        for col in df_temp.columns:
            intermediate_result.loc[row_index, col] = df_temp.loc[df_temp['date'] == now_date, col].values[0]

intermediate_result

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,...,hkd_10,huf_10,jpy_10,krw_10,nok_10,nzd_10,pln_10,sek_10,sgd_10,zar_10
0,2024-12-12,1.564500,1.864300,1.416900,0.888290,23.892000,7.109200,0.953200,0.785700,7.775700,...,7.7828,391.61,151.67,1396.59,11.0643,1.6991,4.0884,10.9443,1.3437,18.1522
1,2024-12-13,1.567600,1.859500,1.421800,0.892280,23.787000,7.089800,0.950750,0.789530,7.775000,...,7.7837,389.84,150.20,1397.57,11.0590,1.6922,4.0674,10.9051,1.3406,18.0626
2,2024-12-16,1.572700,1.863000,1.424900,0.892740,23.848000,7.105000,0.952560,0.790100,7.775300,...,7.7808,394.93,150.13,1404.33,11.0964,1.6981,4.0835,10.9708,1.3456,18.1820
3,2024-12-17,1.576400,1.863200,1.428500,0.896730,23.877000,7.105800,0.952650,0.787350,7.770900,...,7.7833,394.45,149.64,1403.06,11.0626,1.7002,4.0880,11.0117,1.3451,18.1035
4,2024-12-18,1.581700,1.863400,1.431200,0.893860,23.940000,7.108000,0.952740,0.786200,7.770900,...,7.7840,394.96,151.15,1417.54,11.0756,1.7144,4.0858,11.0065,1.3472,18.1738
5,2024-12-19,1.599900,1.881500,1.436300,0.896490,24.172000,7.177800,0.962000,0.793120,7.771500,...,7.7824,390.70,150.40,1418.04,11.0626,1.7064,4.0555,10.9198,1.3409,18.0671
6,2024-12-20,1.604800,1.882400,1.438500,0.894800,24.178000,7.181300,0.962460,0.798510,7.772400,...,7.7794,391.60,150.51,1419.74,11.0996,1.7118,4.0327,10.8903,1.3396,18.0584
7,2024-12-23,1.603700,1.881800,1.441200,0.898300,24.213000,7.176900,0.962190,0.798570,7.770300,...,7.7748,389.15,150.83,1432.14,11.1123,1.7061,4.0304,10.9160,1.3396,17.8194
8,2024-12-24,1.604700,1.881500,1.441800,0.900240,24.180000,7.177300,0.962000,0.796580,7.767500,...,7.7760,390.57,151.74,1431.85,11.1442,1.7178,4.0541,10.9561,1.3411,17.8782
9,2024-12-27,1.607800,1.874300,1.437700,0.900430,24.150000,7.149200,0.958310,0.796340,7.761400,...,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866


In [160]:
predict_exchange = intermediate_result.loc[intermediate_result["date"] > last_date, ["date"] + filtered_columns]
predict_exchange['date'] = pd.to_datetime(predict_exchange['date']).dt.tz_localize(None)
predict_exchange

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,huf,jpy,krw,nok,nzd,pln,sek,sgd,zar
10,2024-12-30,1.541141,1.838320,1.379623,0.897305,24.417318,6.989988,0.950156,0.789757,7.770858,367.428192,148.342514,1355.118408,10.800416,1.702571,4.054369,10.695471,1.360273,18.786877
11,2024-12-31,1.537346,1.851404,1.426810,0.867157,24.537416,7.081013,0.952036,0.792289,7.775136,387.794617,154.219681,1397.775146,10.916081,1.706928,4.084551,10.776754,1.368172,18.659296
12,2025-01-01,1.550713,1.845641,1.381803,0.879927,23.194738,6.919368,0.938662,0.788186,7.767988,360.445984,148.842377,1355.951660,10.594326,1.716984,3.985663,10.477279,1.356640,18.216599
13,2025-01-02,1.608766,1.874190,1.411317,0.876733,24.185942,7.124239,0.946272,0.787828,7.776083,377.691986,153.012772,1380.110718,10.576110,1.702948,4.027237,10.524451,1.389235,18.113506
14,2025-01-03,1.560774,1.871500,1.362973,0.883546,24.057713,6.962848,0.939616,0.792183,7.770098,364.696930,149.622253,1355.974976,10.633256,1.809365,4.001108,10.484788,1.366058,17.927595


In [162]:
df_gold = gold_fg.read().sort_values(by=['date'], ascending=True).tail(20)
df_gold['date'] = pd.to_datetime(df_gold['date']).dt.tz_localize(None)
df_gold

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.51s) 


,date,close_price
874,2024-11-29,2657.000000
4783,2024-12-02,2634.899902
4054,2024-12-03,2644.699951
5903,2024-12-04,2653.800049
902,2024-12-05,2626.600098
170,2024-12-06,2638.600098
4821,2024-12-09,2664.899902
2929,2024-12-10,2697.600098
2531,2024-12-11,2733.800049
2927,2024-12-12,2687.500000


In [180]:
df_exchange = predict_exchange.copy()

for i in range(1, 11):
    df_temp = df_gold.copy()
    df_temp['date'] = pd.to_datetime(df_temp['date']) + BusinessDay(i)
    df_exchange = pd.merge(df_exchange, df_temp[['date','close_price']], how='left', on="date")
    df_exchange = df_exchange.rename(columns={"close_price": f"close_price_{i}"})

df_exchange.fillna(0, inplace=True)

df_exchange

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,huf,jpy,krw,nok,nzd,pln,sek,sgd,zar,close_price_1,close_price_2,close_price_3,close_price_4,close_price_5,close_price_6,close_price_7,close_price_8,close_price_9,close_price_10
0,2024-12-30,1.541141,1.838320,1.379623,0.897305,24.417318,6.989988,0.950156,0.789757,7.770858,367.428192,148.342514,1355.118408,10.800416,1.702571,4.054369,10.695471,1.360273,18.786877,2617.199951,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902
1,2024-12-31,1.537346,1.851404,1.426810,0.867157,24.537416,7.081013,0.952036,0.792289,7.775136,387.794617,154.219681,1397.775146,10.916081,1.706928,4.084551,10.776754,1.368172,18.659296,0.000000,2617.199951,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902
2,2025-01-01,1.550713,1.845641,1.381803,0.879927,23.194738,6.919368,0.938662,0.788186,7.767988,360.445984,148.842377,1355.951660,10.594326,1.716984,3.985663,10.477279,1.356640,18.216599,0.000000,0.000000,2617.199951,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000
3,2025-01-02,1.608766,1.874190,1.411317,0.876733,24.185942,7.124239,0.946272,0.787828,7.776083,377.691986,153.012772,1380.110718,10.576110,1.702948,4.027237,10.524451,1.389235,18.113506,0.000000,0.000000,0.000000,2617.199951,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951
4,2025-01-03,1.560774,1.871500,1.362973,0.883546,24.057713,6.962848,0.939616,0.792183,7.770098,364.696930,149.622253,1355.974976,10.633256,1.809365,4.001108,10.484788,1.366058,17.927595,0.000000,0.000000,0.000000,0.000000,2617.199951,2638.800049,0.000000,2620.000000,2612.300049,2628.699951


In [168]:
# pd.set_option('display.max_rows', None)  # 显示所有行
# pd.set_option('display.max_columns', None)  # 显示所有列

In [184]:
gold_today_df["close_price"][0]

2636.5

In [188]:
for i in range(1, 6):
    date = pd.to_datetime(today) + BusinessDay(i)
    condition = (df_exchange['date'] == date)
    df_exchange.loc[condition, f'close_price_{i}'] = gold_today_df["close_price"][0]

display(df_exchange)
df_exchange.info()

,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,huf,jpy,krw,nok,nzd,pln,sek,sgd,zar,close_price_1,close_price_2,close_price_3,close_price_4,close_price_5,close_price_6,close_price_7,close_price_8,close_price_9,close_price_10
0,2024-12-30,1.541141,1.838320,1.379623,0.897305,24.417318,6.989988,0.950156,0.789757,7.770858,367.428192,148.342514,1355.118408,10.800416,1.702571,4.054369,10.695471,1.360273,18.786877,2636.5,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902
1,2024-12-31,1.537346,1.851404,1.426810,0.867157,24.537416,7.081013,0.952036,0.792289,7.775136,387.794617,154.219681,1397.775146,10.916081,1.706928,4.084551,10.776754,1.368172,18.659296,0.0,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902
2,2025-01-01,1.550713,1.845641,1.381803,0.879927,23.194738,6.919368,0.938662,0.788186,7.767988,360.445984,148.842377,1355.951660,10.594326,1.716984,3.985663,10.477279,1.356640,18.216599,0.0,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000
3,2025-01-02,1.608766,1.874190,1.411317,0.876733,24.185942,7.124239,0.946272,0.787828,7.776083,377.691986,153.012772,1380.110718,10.576110,1.702948,4.027237,10.524451,1.389235,18.113506,0.0,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951
4,2025-01-03,1.560774,1.871500,1.362973,0.883546,24.057713,6.962848,0.939616,0.792183,7.770098,364.696930,149.622253,1355.974976,10.633256,1.809365,4.001108,10.484788,1.366058,17.927595,0.0,0.000000,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            5 non-null      datetime64[us]
 1   aud             5 non-null      float64       
 2   bgn             5 non-null      float64       
 3   cad             5 non-null      float64       
 4   chf             5 non-null      float64       
 5   czk             5 non-null      float64       
 6   dkk             5 non-null      float64       
 7   eur             5 non-null      float64       
 8   gbp             5 non-null      float64       
 9   hkd             5 non-null      float64       
 10  huf             5 non-null      float64       
 11  jpy             5 non-null      float64       
 12  krw             5 non-null      float64       
 13  nok             5 non-null      float64       
 14  nzd             5 non-null      float64       
 15  pln       

In [189]:
display(gold_fg.read().sort_values(by=['date'], ascending=True).tail(10))
display(exchange_fg.read().sort_values(by=['date'], ascending=True).tail(10))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.12s) 


,date,close_price
2935,2024-12-13 00:00:00+00:00,2656.000000
5357,2024-12-16 00:00:00+00:00,2651.399902
3292,2024-12-17 00:00:00+00:00,2644.399902
2607,2024-12-18 00:00:00+00:00,2636.500000
5078,2024-12-19 00:00:00+00:00,2592.199951
5611,2024-12-20 00:00:00+00:00,2628.699951
1922,2024-12-23 00:00:00+00:00,2612.300049
1809,2024-12-24 00:00:00+00:00,2620.000000
6102,2024-12-26 00:00:00+00:00,2638.800049
6103,2024-12-27 00:00:00+00:00,2617.199951


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.43s) 


,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,huf,jpy,krw,nok,nzd,pln,sek,sgd,zar,close_price_1,close_price_2,close_price_3,close_price_4,close_price_5,close_price_6,close_price_7,close_price_8,close_price_9,close_price_10
2997,2024-12-12 00:00:00+00:00,1.5645,1.8643,1.4169,0.88829,23.892,7.1092,0.95320,0.78570,7.7757,389.33,152.13,1430.72,11.1086,1.7266,4.0757,10.9618,1.3448,17.6939,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902,2657.000000,0.000000
3004,2024-12-13 00:00:00+00:00,1.5676,1.8595,1.4218,0.89228,23.787,7.0898,0.95075,0.78953,7.7750,388.76,153.50,1433.33,11.1008,1.7321,4.0543,10.9479,1.3472,17.8866,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902,2657.000000
5469,2024-12-16 00:00:00+00:00,1.5727,1.8630,1.4249,0.89274,23.848,7.1050,0.95256,0.79010,7.7753,389.63,154.06,1437.56,11.1602,1.7326,4.0617,10.9107,1.3501,17.8788,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951,2634.899902
3371,2024-12-17 00:00:00+00:00,1.5764,1.8632,1.4285,0.89673,23.877,7.1058,0.95265,0.78735,7.7709,390.06,153.98,1436.72,11.2065,1.7366,4.0616,10.9288,1.3510,18.1286,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049,2644.699951
2673,2024-12-18 00:00:00+00:00,1.5817,1.8634,1.4312,0.89386,23.940,7.1080,0.95274,0.78620,7.7709,391.46,153.89,1438.15,11.2062,1.7438,4.0566,10.9466,1.3507,18.0642,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098,2653.800049
5183,2024-12-19 00:00:00+00:00,1.5999,1.8815,1.4363,0.89649,24.172,7.1778,0.96200,0.79312,7.7715,400.24,156.87,1448.10,11.3654,1.7688,4.0885,11.0269,1.3594,18.3117,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098
5730,2024-12-20 00:00:00+00:00,1.6048,1.8824,1.4385,0.89480,24.178,7.1813,0.96246,0.79851,7.7724,398.63,156.78,1446.29,11.4230,1.7753,4.1023,11.0452,1.3571,18.3931,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098
1975,2024-12-23 00:00:00+00:00,1.6037,1.8818,1.4412,0.89830,24.213,7.1769,0.96219,0.79857,7.7703,399.02,157.18,1453.44,11.3605,1.7732,4.1136,11.0497,1.3570,18.4780,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902
1862,2024-12-24 00:00:00+00:00,1.6047,1.8815,1.4418,0.90024,24.180,7.1773,0.96200,0.79658,7.7675,396.08,157.05,1458.37,11.3987,1.7729,4.1092,11.0952,1.3592,18.5948,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098
2423,2024-12-27 00:00:00+00:00,1.6078,1.8743,1.4377,0.90043,24.150,7.1492,0.95831,0.79634,7.7614,394.30,157.79,1473.55,11.3498,1.7736,4.0971,11.0010,1.3583,18.7119,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000


In [190]:
def insert_data(feature_group, data):
    return feature_group.insert(data, write_options={"wait_for_job": True})

with concurrent.futures.ThreadPoolExecutor() as executor:
    future1 = executor.submit(insert_data, gold_fg, gold_today_df)
    future2 = executor.submit(insert_data, exchange_fg, df_exchange)

    concurrent.futures.wait([future1, future2])


2024-12-29 23:32:39,310 INFO: 	1 expectation(s) included in expectation_suite.
Validation failed.
2024-12-29 23:32:39,565 INFO: 	28 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1393256
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1393338


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00
Uploading Dataframe: 0.00% |          | Rows 0/5 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: gold_feature_group_1_offline_fg_materialization


Uploading Dataframe: 100.00% |██████████| Rows 5/5 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: exchange_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1163416/jobs/named/gold_feature_group_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1163416/jobs/named/exchange_feature_group_1_offline_fg_materialization/executions
2024-12-29 23:32:54,246 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-29 23:32:55,560 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-29 23:32:57,413 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-29 23:32:58,717 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-29 23:34:20,916 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-29 23:34:21,061

In [191]:
display(gold_fg.read().sort_values(by=['date'], ascending=True).tail(10))
display(exchange_fg.read().sort_values(by=['date'], ascending=True).tail(10))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.29s) 


,date,close_price
2935,2024-12-13 00:00:00+00:00,2656.000000
5357,2024-12-16 00:00:00+00:00,2651.399902
3292,2024-12-17 00:00:00+00:00,2644.399902
2607,2024-12-18 00:00:00+00:00,2636.500000
5078,2024-12-19 00:00:00+00:00,2592.199951
5611,2024-12-20 00:00:00+00:00,2628.699951
1922,2024-12-23 00:00:00+00:00,2612.300049
1809,2024-12-24 00:00:00+00:00,2620.000000
6102,2024-12-26 00:00:00+00:00,2638.800049
6103,2024-12-27 00:00:00+00:00,2636.500000


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.39s) 


,date,aud,bgn,cad,chf,czk,dkk,eur,gbp,hkd,huf,jpy,krw,nok,nzd,pln,sek,sgd,zar,close_price_1,close_price_2,close_price_3,close_price_4,close_price_5,close_price_6,close_price_7,close_price_8,close_price_9,close_price_10
5183,2024-12-19 00:00:00+00:00,1.599900,1.881500,1.436300,0.896490,24.172000,7.177800,0.962000,0.793120,7.771500,400.240000,156.870000,1448.100000,11.365400,1.768800,4.088500,11.026900,1.359400,18.311700,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098,2626.600098
5730,2024-12-20 00:00:00+00:00,1.604800,1.882400,1.438500,0.894800,24.178000,7.181300,0.962460,0.798510,7.772400,398.630000,156.780000,1446.290000,11.423000,1.775300,4.102300,11.045200,1.357100,18.393100,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902,2638.600098
1975,2024-12-23 00:00:00+00:00,1.603700,1.881800,1.441200,0.898300,24.213000,7.176900,0.962190,0.798570,7.770300,399.020000,157.180000,1453.440000,11.360500,1.773200,4.113600,11.049700,1.357000,18.478000,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098,2664.899902
1862,2024-12-24 00:00:00+00:00,1.604700,1.881500,1.441800,0.900240,24.180000,7.177300,0.962000,0.796580,7.767500,396.080000,157.050000,1458.370000,11.398700,1.772900,4.109200,11.095200,1.359200,18.594800,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000,2687.500000,2733.800049,2697.600098
2423,2024-12-27 00:00:00+00:00,1.607800,1.874300,1.437700,0.900430,24.150000,7.149200,0.958310,0.796340,7.761400,394.300000,157.790000,1473.550000,11.349800,1.773600,4.097100,11.001000,1.358300,18.711900,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902,2656.000000
6230,2024-12-30 00:00:00+00:00,1.541141,1.838320,1.379623,0.897305,24.417318,6.989988,0.950156,0.789757,7.770858,367.428192,148.342514,1355.118408,10.800416,1.702571,4.054369,10.695471,1.360273,18.786877,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902,2651.399902
6231,2024-12-31 00:00:00+00:00,1.537346,1.851404,1.426810,0.867157,24.537416,7.081013,0.952036,0.792289,7.775136,387.794617,154.219681,1397.775146,10.916081,1.706928,4.084551,10.776754,1.368172,18.659296,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000,2644.399902
6229,2025-01-01 00:00:00+00:00,1.550713,1.845641,1.381803,0.879927,23.194738,6.919368,0.938662,0.788186,7.767988,360.445984,148.842377,1355.951660,10.594326,1.716984,3.985663,10.477279,1.356640,18.216599,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951,2636.500000
6228,2025-01-02 00:00:00+00:00,1.608766,1.874190,1.411317,0.876733,24.185942,7.124239,0.946272,0.787828,7.776083,377.691986,153.012772,1380.110718,10.576110,1.702948,4.027237,10.524451,1.389235,18.113506,0.000000,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951,2592.199951
6232,2025-01-03 00:00:00+00:00,1.560774,1.871500,1.362973,0.883546,24.057713,6.962848,0.939616,0.792183,7.770098,364.696930,149.622253,1355.974976,10.633256,1.809365,4.001108,10.484788,1.366058,17.927595,0.000000,0.000000,0.000000,0.000000,2636.500000,2638.800049,0.000000,2620.000000,2612.300049,2628.699951
